In [ ]:
#!nvidia-smi
#!wget -c "https://raw.githubusercontent.com/chef-transformer/chef-transformer/main/data/test-5000.csv" /content/
#%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1
import numpy as np
import pandas as pd

from tqdm import tqdm

import os
import json
import re
import textwrap

import torch
import torch.nn.functional as F
import evaluate

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
model_name_or_path = "mbien/recipenlg"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(model_name_or_path, device)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name_or_path).to(device)
dataset = load_dataset("csv", data_files={"test": "/content/test-5000.csv"}, delimiter="\t")["test"]
dataset
tokenizer("<RECIPE_START>")
def post_processing(text):
    r_s, r_e = "<RECIPE_START>", "<RECIPE_END>"
    t_s, t_e = "<TITLE_START>", "<TITLE_END>"
    i_s, i_e = "<INGR_START>", "<INGR_END>"
    d_s, d_e = "<INSTR_START>", "<INSTR_END>"

    if not r_s in text or not r_e in text:
        return None
    
    text = text[text.find(r_s) + len(r_s): text.find(r_e)]

    if not t_s in text or not i_s in text or not d_s in text:
        return None
    
    if not t_e in text or not i_e in text or not d_e in text:
        return None

    title = text[text.find(t_s) + len(t_s): text.find(t_e)]
    ingredients = text[text.find(i_s) + len(i_s): text.find(i_e)].replace("<NEXT_INGR>", " ")
    directions = text[text.find(d_s) + len(d_s): text.find(d_e)].replace("<NEXT_INSTR>", " ")

    if not len(title) > 0 or not len(ingredients) > 0 or not len(directions) > 0:
        return None

    title = [w.strip() for w in title.split() if w.strip()]
    ingredients = [w.strip() for w in ingredients.split() if w.strip()]
    directions = [w.strip() for w in directions.split() if w.strip()]

    title = ' '.join(title).lower()
    ingredients = ' '.join(ingredients).lower()
    directions = ' '.join(directions).lower()

    return f"title: {title} ingredients: {ingredients} directions: {directions}"


def post_processing_list(texts):
    new_texts = []
    
    for text in texts:
        text = post_processing(text)
        if text:
            new_texts.append(text)
    
    return new_texts

def predict(batch):
    inputs = [
        '<RECIPE_START> <INPUT_START> ' + inp.replace(', ', ' <NEXT_INPUT> ') + ' <INPUT_END>'
        for inp in batch["inputs"]
    ]
    
    inputs = tokenizer(
        inputs, 
        max_length=256, 
        padding=True, 
        truncation=True, 
        return_tensors='pt'
    )
    input_ids = inputs.input_ids.to(device)
    # attention_mask = inputs.attention_mask.to(device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids, 
            # attention_mask=attention_mask,
            **genkw
        )

    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=False)
    batch["predicted"] = [post_processing_list(outputs[i:i + genkw["num_return_sequences"]]) for i in range(0, len(outputs), genkw["num_return_sequences"])]
    return batch
sample = dataset.select(range(1))
result = dataset.map(predict, batched=True, batch_size=8)
result
df_list = []
for i in tqdm(range(len(result)), position=0):
    inputs, targets, predicted =  result["inputs"][i], result["targets"][i], result["predicted"][i]
    df_list.append({
        "input": inputs,
        "target": targets,
        "predicted": predicted,
    })

df = pd.DataFrame(df_list)
df.to_json("/content/predicted.json")
df.head()
df = pd.read_json("/content/predicted.json")
print(df.shape)
df["predicted"] = df["predicted"].apply(lambda t: t if len(t) > 0 else None)
df = df.dropna()
df = df.reset_index(drop=True) 
print(df.shape)
df.head()
!pip freeze | grep nltk
%%capture
!pip install -U nltk
!pip install jiwer
!pip install rouge_score
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

import re
import math
from collections import Counter
import string
from tqdm import tqdm
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = stopwords.words("english")
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
        
def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)

def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b
    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)
    cosine_result = get_cosine(vector1, vector2)
    return cosine_result
cosim_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted_list = row["target"], row["predicted"]

    target = " ".join([word.strip() for word in wordpunct_tokenize(target) if word not in string.punctuation and word not in stopwords and word])

    cosim_score = []
    for j in range(len(predicted_list)):
        predicted = " ".join([word.strip() for word in wordpunct_tokenize(predicted_list[j]) if word not in string.punctuation and word not in stopwords and word])
        cosim_score.append(get_result(target, predicted))

    cosim_scores.append(max(cosim_score))
    # break
    cosim_score = np.array(cosim_scores).mean()
print(f"Cosine similarity score: {cosim_score}")
wer = load_metric("wer")
wer_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted_list = row["target"], row["predicted"]

    wer_score = []
    for j in range(len(predicted_list)):
        score = wer.compute(references=[target], predictions=[predicted_list[j]])
        wer_score.append(score)

    wer_scores.append(min(wer_score))

    # break
    wer_score = np.array(wer_scores).mean()
print(f"WER score: {wer_score}")
rouge = load_metric("rouge")
rouge_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predictions = row["target"], row["predicted"]

    references = [target] * len(predictions)
    score = rouge.compute(references=references, predictions=predictions, use_stemmer=True)
    rouge2 = score["rouge2"].high.fmeasure
    rouge_scores.append(rouge2)
    rouge_score = np.array(rouge_scores).mean()
print(f"Rouge score: {rouge_score}") 
import nltk
import nltk.translate.bleu_score as bleu
from nltk.translate.bleu_score import SmoothingFunction

import nltk.translate.gleu_score as gleu
import nltk.translate.meteor_score as meteor


def compute_bleu(recipe, refer):
    hyp = recipe
    refs = refer
    smoothie = SmoothingFunction().method4
    score_ref_a = bleu.sentence_bleu(refs, hyp, smoothing_function=smoothie)
    return score_ref_a


def compute_gleu(recipe, refer):
    hyp = recipe
    refs = refer
    score_ref_a = gleu.sentence_gleu(refs, hyp)
    return score_ref_a

def compute_meteor(recipe, refer):
    hyp = recipe
    refs = refer
    score_ref_a = meteor.meteor_score(refs, hyp)
    return score_ref_a
bleu_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted = row["target"], row["predicted"]

    target = wordpunct_tokenize(target)
    predicted = [wordpunct_tokenize(p) for p in predicted]

    score = compute_bleu(target, predicted)
    bleu_scores.append(score)

    # break
    bleu_score = np.array(bleu_scores).mean()
print(f"BLEU score: {bleu_score}")
gleu_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted = row["target"], row["predicted"]

    target = wordpunct_tokenize(target)
    predicted = [wordpunct_tokenize(p) for p in predicted]

    score = compute_gleu(target, predicted)
    gleu_scores.append(score)

    # break
    gleu_score = np.array(gleu_scores).mean()
print(f"GLEU score: {gleu_score}")
m_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted = row["target"], row["predicted"]

    score = compute_meteor(target, predicted)
    m_scores.append(score)

    # break
    m_score = np.array(m_scores).mean()
print(f"METEOR score: {m_score}")

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-j790fogv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-j790fogv
  Resolved https://github.com/huggingface/datasets.git to commit 01f91bae037c98f2e05456287bab21470adb8f07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-uhhah71j
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-uhhah71j
  Resolved https://github.com/huggingface/transformers.git to commit 7bbc62474391aff64f63fcc064c975752d1fa4de
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tokenizers-0.20.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86

/home/dark/miniconda3/envs/ganesh/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'load_metric' from 'datasets' (/home/dark/miniconda3/envs/ganesh/lib/python3.8/site-packages/datasets/__init__.py)

In [2]:
pip install evaluate


Note: you may need to restart the kernel to use updated packages.
